# Example 1.5: Spacefilling Curve
### Zack Kenyon, 2024

The curve below is part of a family of $C^2$, nonself-intersecting curves whose limit is surjective on the equilateral triangle. 

In [ ]:
import puncturedfem as pf
import numpy as np

In [ ]:
family_param = 3

In [ ]:
# accepts the vertices of a triangle to find the coordinates of a space filling curve
def SFC(A, B, C, n, p):
    # switch on the top 2 bits of n.
    if p == 0:
        return [
            2 * A / 3 + (A + B + C) / 9,
            2 * (A / 6 + 2 * B / 3 + C / 6) / 3 + (A + B + C) / 9,
            2 * C / 3 + (A + B + C) / 9,
        ]

    switch = n >> 2 * (p - 1) & 3

    if switch == 0:
        return SFC(A, (A + C) / 2, (A + B) / 2, n, p - 1)
    elif switch == 1:
        return SFC((A + B) / 2, B, (C + B) / 2, n, p - 1)
    elif switch == 2:
        return SFC((B + C) / 2, (A + B) / 2, (A + C) / 2, n, p - 1)
    elif switch == 3:
        return SFC((A + C) / 2, (B + C) / 2, C, n, p - 1)


idxs = np.arange(3 * 4**family_param)
A = np.array([0, 0])
B = np.array([1.0 / 2, np.sqrt(3) / 2])
C = np.array([1, 0])
D = (A + B + C) / 3
A_ = (1 - 1 / 4**family_param) * A + 1 / 4**family_param * D
B_ = (1 - 1 / 4**family_param) * B + 1 / 4**family_param * D
C_ = (1 - 1 / 4**family_param) * C + 1 / 4**family_param * D


def myfun_x(n):
    return SFC(A_, B_, C_, n // 3, family_param)[n % 3][0]


def myfun_y(n):
    return SFC(A_, B_, C_, n // 3, family_param)[n % 3][1]


xs = np.append(
    [0, A_[0]], np.append(np.array([myfun_x(n) for n in idxs]), [C_[0], 1])
)
ys = np.append([0, 0], np.append(np.array([myfun_y(n) for n in idxs]), [0, 0]))

## Construct the mesh cell


In [ ]:
vA = pf.Vert(0, 0, 0)
vB = pf.Vert(1 / 2, np.sqrt(3) / 2, 1)
vC = pf.Vert(1, 0, 2)
badedge = pf.Edge(vA, vC, curve_type="spline", pos_cell_idx=0, pts=[xs, ys])


def Bd(edge, n, l, r):
    if n <= 0:
        return [edge]
    e0, e1 = pf.split_edge(e=edge, t_split=(l + r) / 2)
    return Bd(e0, n - 1, l, (l + r) / 2) + Bd(e1, n - 1, (l + r) / 2, r)


edges = Bd(badedge, 2 * (family_param - 1), 0, 2 * np.pi) + [
    pf.Edge(vC, vB, pos_cell_idx=0, idx=1),
    pf.Edge(vB, vA, pos_cell_idx=0, idx=2),
]

With the edges defined, let's make the mesh cell $K$.

In [ ]:
K = pf.MeshCell(idx=0, edges=edges)

Let's parameterize the edges, and plot the edges to check that we have what we 
want. These curves are a little too close to the boundary, and we can fix that easily, but we'll ignore it for the moment

In [ ]:
quad_dict = pf.get_quad_dict(n=64, p=7)
K.parameterize(quad_dict, compute_interior_points=True)
pf.plot.MeshPlot(K.get_edges()).draw()

In [ ]:
max_curvature = 0
for contour in K.components:
    for edge in contour.edges:
        max_curvature = max(max_curvature, max(edge.curvature))
print(f"{max_curvature:.4e}")

## Construct a basis of the local Poisson space
Since we have subdivided the bad edge, we must build our *bad* basis functions for $V_p^{\partial K}$ by hand. 

In [ ]:
z0 = vA.get_coord_array()
z1 = vC.get_coord_array()
z2 = vB.get_coord_array()
print(z0, z1, z2)

bc = pf.barycentric_coordinates(z0, z1, z2)
for poly in bc:
    print(poly)

In [ ]:
nyst = pf.NystromSolver(K, verbose=True)

In [ ]:
basis: list[pf.LocalPoissonFunction] = []

# vertex functions
for j in range(3):
    trace = pf.DirichletTrace(edges=K.get_edges(), funcs=bc[j])
    phi = pf.LocalPoissonFunction(nyst=nyst, trace=trace)
    basis.append(phi)

# edge function trace is the trace of the barycentric coordinate of vertex vB,
# on all edges except 2
edge_fun_trace = pf.DirichletTrace(edges=K.get_edges(), funcs=bc[2])
straight_edge_indices = [K.num_edges - 2, K.num_edges - 1]
for edge_idx in straight_edge_indices:
    edge_fun_trace.set_trace_values_on_edge(edge_index=edge_idx, values=0.0)

# add edge function to the basis
phi = pf.LocalPoissonFunction(nyst=nyst, trace=edge_fun_trace)
basis.append(phi)

## Plot the basis functions 

In [ ]:
for j, phi in enumerate(basis):
    pf.plot.TracePlot(
        traces=phi.get_trace_values(),
        K=K,
        quad_dict=quad_dict,
        title=f"phi_{j} trace",
    ).draw()
    pf.plot.LocalFunctionPlot(phi).draw(title=f"phi_{j} interior values")

## Compute the local stiffness matrix

In [ ]:
mat = np.zeros((4, 4))
for i, vi in enumerate(basis):
    for j, vj in enumerate(basis):
        mat[i][j] = vi.get_h1_semi_inner_prod(vj)
print(mat)

# ensure mat is perfectly symmetric
mat = (mat + np.transpose(mat)) / 2

# compute the eigenvalues
eigvals = np.linalg.eigvalsh(mat)
print(eigvals)

# ratio between smallest and largest nonzero eigenvalues
lam_min = eigvals[1]
lam_max = eigvals[-1]
ratio = lam_max / lam_min
print(f"family_param = {family_param}")
print(f"lam_min = {lam_min:.4e}")
print(f"lam_max = {lam_max:.4e}")
print(f"ratio = {ratio:.4e}")

|family_param|lam_min   |lam_max   |ratio     |
|------------|----------|----------|----------|
|1           |4.8512e-01|2.3926e+00|4.9320e+00|
|2           |4.3987e-01|1.0488e+01|2.3844e+01|
|3           |4.3339e-01|2.9451e+01|6.7955e+01|